In [1]:
import pandas as pd
pd.set_option("display.max_columns", 100)
pd.set_option("display.max_rows", 100)
import numpy as np
import os
from utils import peek
import config

## 加载样本数据

In [2]:
sample = pd.read_csv("/sda/huweipeng/project/长安银行/data/third_party_data/百融数据/百融样本.csv", parse_dates=['申请日期'])
label = pd.read_csv("/sda/huweipeng/project/长安银行/data/third_party_data/百融数据/样本核卡状态.csv", parse_dates=['apply_date_key'])
sample = sample.merge(label, left_on=['申请日期', '手机号'], right_on=['apply_date_key', 'customer_phone'], how = 'left')[['customer_phone', 'apply_date_key', 'order_status_key']]
sample = sample.dropna(subset=['order_status_key']).drop_duplicates()
sample.columns = pd.MultiIndex.from_product([['sample'], sample.columns])
peek(sample, 2)

(10006, 3)
                             sample                                
                     customer_phone apply_date_key order_status_key
0  cc92befb0e0b9f7ef4736f0fabc86821     2023-02-22                3
1  057331edd64b9002ddfc084e2d09b441     2023-03-25                2


In [3]:
def parse_br_file(path):
    df = pd.read_csv(path, header=[0, 1])
    df.columns = df.columns.get_level_values(0)
    df['user_date'] = pd.to_datetime(df['user_date'])
    return df

def merge_br_features(df, br_features, feature_index):
    customer = pd.read_csv("/sda/huweipeng/project/长安银行/data/third_party_data/百融数据/百融样本.csv", parse_dates=['申请日期'])
    br_features['customer_phone'] = customer['手机号']
    br_features.columns = pd.MultiIndex.from_product([[feature_index], br_features.columns])
    df = df.merge(br_features, left_on=[('sample', 'customer_phone'), ('sample', 'apply_date_key')], 
                          right_on=[(feature_index, 'customer_phone'), (feature_index, 'user_date')], how='left').drop_duplicates()
    df = df.drop([(feature_index, c) for c in ['cus_num', 'user_date', 'swift_number', 'customer_phone']], axis=1)
    return df

## 加载百融多头特征

In [4]:
br_duotou = parse_br_file("/sda/huweipeng/project/长安银行/data/third_party_data/百融数据/样本量10000/详细匹配数据及字典/借贷意向验证-V2.0.csv")
sample = merge_br_features(sample, br_duotou, 'br_duotou')
# customer = pd.read_csv("/sda/huweipeng/project/长安银行/data/third_party_data/百融数据/百融样本.csv", parse_dates=['申请日期'])
# br_duotou['customer_phone'] = customer['手机号']
# br_duotou = br_duotou[br_duotou['flag_applyloanstr'] == 1]
# br_duotou.columns = pd.MultiIndex.from_product([['br_duotou'], br_duotou.columns])
# # use_features = pd.read_csv("/sda/huweipeng/project/长安银行/data/third_party_data/feature_iv/百融多头iv.csv")
# # use_features = use_features['feature'].tolist() + ['customer_phone', '申请日期']
# # br_duotou = br_duotou[use_features]
# # special_features = ['按身份证号查询，近3个月在非银机构申请最小间隔天数',
# #                      '按身份证号查询，近12个月在非银机构申请最小间隔天数',
# #                      '按身份证号查询，近12个月在非银机构周末申请机构数',
# #                      '按身份证号查询，近12个月申请最小间隔天数',
# #                      '按身份证号查询，近1个月在非银机构周末申请机构数',
# #                      '按身份证号查询，近6个月在银行机构申请最小间隔天数',
# #                      '按身份证号查询，近7天在非银机构周末申请机构数',
# #                      '按身份证号查询，近12个月在银行机构申请最小间隔天数']
# # normal_features = br_duotou.columns.difference(special_features)
# # br_duotou[special_features] = br_duotou[special_features].fillna(9999)
# # br_duotou[normal_features] = br_duotou[normal_features].fillna(0)
# # 将字段转化为英文，方便数据处理
# sample = sample.merge(br_duotou, left_on=[('sample', 'customer_phone'), ('sample', 'apply_date_key')], 
#                       right_on=[('br_duotou', 'customer_phone'), ('br_duotou', 'user_date')], how='left').drop_duplicates()
# sample = sample.drop([('br_duotou', c) for c in ['cus_num', 'user_date', 'swift_number', 'customer_phone']], axis=1)

In [5]:
peek(sample)

(10047, 824)
                             sample                                  \
                     customer_phone apply_date_key order_status_key   
0  cc92befb0e0b9f7ef4736f0fabc86821     2023-02-22                3   
1  057331edd64b9002ddfc084e2d09b441     2023-03-25                2   
2  2d84b1d936cfc398062267ae3c88ba5e     2023-03-22                2   
3  524900208eb2c8843ced18b6c34eee4a     2023-03-22                3   
4  66fcdc3a4b9f45ef280f28853a5a007d     2023-03-20                3   

          br_duotou                                            \
  flag_applyloanstr als_d7_id_pdl_allnum als_d7_id_pdl_orgnum   
0                 1                  NaN                  NaN   
1                 1                  NaN                  NaN   
2                 1                  NaN                  NaN   
3                 1                  NaN                  NaN   
4                 1                  NaN                  NaN   

                                 

## 加载百融黑名单

In [6]:
br_black_list = parse_br_file("/sda/huweipeng/project/长安银行/data/third_party_data/百融数据/样本量10000/详细匹配数据及字典/特殊名单验证-V2.1.csv")
sample = merge_br_features(sample, br_black_list, 'br_black_list')
# br_black_list['customer_phone'] = customer['手机号']
# br_black_list.columns = pd.MultiIndex.from_product([['br_black_list'], br_black_list.columns])
# sample = sample.merge(br_black_list, 
#                       left_on=[('sample', 'customer_phone'), ('sample', 'apply_date_key')], 
#                       right_on=[('br_black_list', 'customer_phone'), ('br_black_list', 'user_date')], how='left').drop_duplicates()
# sample = sample.drop([('br_black_list', c) for c in ['cus_num', 'user_date', 'swift_number', 'customer_phone']], axis=1)

## 加载百融偿债压力指数

In [7]:
br_debt_pressure = parse_br_file("/sda/huweipeng/project/长安银行/data/third_party_data/百融数据/样本量10000/详细匹配数据及字典/偿债压力指数-V1.0.csv")
sample = merge_br_features(sample, br_debt_pressure, 'br_debt_pressure')
# br_debt_pressure['customer_phone'] = customer['手机号']
# br_debt_pressure.columns = pd.MultiIndex.from_product([['br_debt_pressure'], br_debt_pressure.columns])
# sample = sample.merge(br_debt_pressure, left_on=[('sample', 'customer_phone'), ('sample', 'apply_date_key')], 
#                       right_on=[('br_debt_pressure', 'customer_phone'), ('br_debt_pressure', 'user_date')], 
#                       how='left').drop_duplicates()
# # sample.rename(columns={'偿债压力指数':'debt_pressure_index'}, inplace=True)
# sample = sample.drop([('br_debt_pressure', c) for c in ['cus_num', 'user_date', 'swift_number', 'customer_phone']], axis=1)

## 加载百融法院被执行人

In [8]:
br_credit_broken = parse_br_file("/sda/huweipeng/project/长安银行/data/third_party_data/百融数据/样本量10000/详细匹配数据及字典/法院被执行人—高级版-V2.0.csv")
sample = merge_br_features(sample, br_credit_broken, 'br_execution')
# br_credit_broken['customer_phone'] = customer['手机号']

# sample = sample.merge(br_credit_broken[['customer_phone', 'user_date', 'executionpro']], left_on=['customer_phone', 'apply_date_key'], right_on=['customer_phone', 'user_date'], 
#                       how='left')
# sample = sample.drop(columns=['user_date']).drop_duplicates()

/tmp/ipykernel_159090/4164070112.py:2: DtypeWarning: Columns (35,82,98,99,100,101,102,103,104,105,106,107,110,112,114,129,131,136,145,146,147,148,149,150,151,152,153,154,157,159,161,172,176,177,178,179,180,181,182,183,184,190,219,224,225,226,227,228,229,230,231,237,266,271,272,273,274,275,276,277,278,284,304,305,307,308,309,313,315,318,319,320,321,322,323,324,325,331,351,352,354,355,356,362,365,366,367,368,369,370,371,372,378,398,399,401,402,403,407,409,445,446,448,449,450,454,456) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(path, header=[0, 1])


## 加载百融停机次数

In [9]:
br_cell_disconnect = parse_br_file("/sda/huweipeng/project/长安银行/data/third_party_data/百融数据/样本量1000/详细匹配数据及字典/手机3个月停机次数验证.csv")
sample = merge_br_features(sample, br_cell_disconnect, "br_cell_disconnect")
# br_cell_disconnect['customer_phone'] = customer['手机号']
# br_cell_disconnect = br_cell_disconnect[br_cell_disconnect['flag_teldownnum'] == 1]
# br_cell_disconnect.rename(columns={'result':'disconnect_times', 'operation':'disconnect_operation'}, inplace=True)
# sample = sample.merge(br_cell_disconnect[['customer_phone', 'user_date', 'disconnect_operation', 'disconnect_times']], left_on=['customer_phone', 'apply_date_key'], 
#                       right_on=['customer_phone', 'user_date']).drop_duplicates()
# sample = sample.drop(['user_date'], axis=1).drop_duplicates()
# # print(br_cell_disconnect.shape)
# peek(br_cell_disconnect, 10)

## 加载百融在网时长

In [10]:
br_cell_length = parse_br_file("/sda/huweipeng/project/长安银行/data/third_party_data/百融数据/样本量1000/详细匹配数据及字典/手机在网时长-移动联通电信.csv")
sample = merge_br_features(sample, br_cell_length, 'br_cell_length')
# br_cell_length['customer_phone'] = customer['手机号']
# br_cell_length = br_cell_length[br_cell_length['flag_telperiod'] == 1]
# br_cell_length.rename(columns={'operation':'cell_length_operation', 'value':'cell_length'}, inplace=True)
# sample = sample.merge(br_cell_length[['customer_phone', 'user_date', 'cell_length_operation', 'cell_length']], 
#                       left_on=['customer_phone', 'apply_date_key'], right_on=['customer_phone', 'user_date']).drop_duplicates()
# sample = sample.drop(['user_date'], axis=1).drop_duplicates()

## 加载百融在网状态

In [11]:
br_cell_status = parse_br_file("/sda/huweipeng/project/长安银行/data/third_party_data/百融数据/样本量1000/详细匹配数据及字典/手机在网状态-移动联通电信.csv")
sample = merge_br_features(sample, br_cell_status, 'br_cell_status')
# br_cell_status['customer_phone'] = customer['手机号']
# br_cell_status = br_cell_status[br_cell_status['flag_telstatus'] == 1]
# br_cell_status.rename(columns={'operation':'cell_status_operation', 'value':'cell_status'}, inplace=True)
# sample = sample.merge(br_cell_status[['customer_phone', 'user_date', 'cell_status_operation', 'cell_status']]
#                       , left_on=['customer_phone', 'apply_date_key'], right_on=['customer_phone', 'user_date']).drop_duplicates()
# sample = sample.drop(['user_date'], axis=1).drop_duplicates()

## 加载腾讯反欺诈

In [12]:
def merge_tencent_features(df, tencent_features, feature_index):
    tencent_features.columns = pd.MultiIndex.from_product([[feature_index], tencent_features.columns])
    df = df.merge(tencent_features, right_on=[(feature_index, '手机号'), (feature_index, '回溯时间')], 
                  left_on=[('sample', 'customer_phone'), ('sample', 'apply_date_key')], how='left')
    df = df.drop(columns=[(feature_index, c) for c in ['身份证号', '姓名', '手机号', '回溯时间']], axis=1).drop_duplicates()
    return df

In [13]:
tencent_fraud_risk = pd.read_excel("/sda/huweipeng/project/长安银行/data/third_party_data/腾讯数据/数字魔方_反欺诈v5v6v7-zx&灵鲲v6&行业风险3.0_20230406.xlsx", 
                      header = 1, parse_dates=['回溯时间'], sheet_name="反欺诈")
sample = merge_tencent_features(sample, tencent_fraud_risk, 'tc_fraud_risk')
# tencent_fraud_risk = tencent_fraud_risk.dropna(subset=['highirr_v6_20220425_score'])
# # tencent_fraud_risk = tencent_fraud_risk[use_feature]
# sample = sample.merge(tencent_fraud_risk, right_on=['手机号', '回溯时间'], left_on=['customer_phone', 'apply_date_key'], how='left')
# sample = sample.drop(columns=['身份证号', '姓名', '手机号', '回溯时间', 'id_found:', 'found:', 'risk_code:']).drop_duplicates()

## 加载腾讯多头

In [14]:
tencent_duotou = pd.read_excel("/sda/huweipeng/project/长安银行/data/third_party_data/腾讯数据/数字魔方_反欺诈v5v6v7-zx&灵鲲v6&行业风险3.0_20230406.xlsx", 
                        parse_dates=['回溯时间'], sheet_name="行业风险-多头申请v3")
sample = merge_tencent_features(sample, tencent_duotou, 'tc_duotou')

## 加载腾讯逾期

In [15]:
tencent_overdue = pd.read_excel("/sda/huweipeng/project/长安银行/data/third_party_data/腾讯数据/数字魔方_反欺诈v5v6v7-zx&灵鲲v6&行业风险3.0_20230406.xlsx", 
                        parse_dates=['回溯时间'], sheet_name="行业风险-逾期")
sample = merge_tencent_features(sample, tencent_overdue, 'tc_overdue')

## 加载腾讯圈团&可疑&电诈

In [16]:
tencent_risk = pd.read_excel("/sda/huweipeng/project/长安银行/data/third_party_data/腾讯数据/数字魔方_反欺诈v5v6v7-zx&灵鲲v6&行业风险3.0_20230406.xlsx", 
                        parse_dates=['回溯时间'], sheet_name="行业风险-圈团&可疑&电诈")
sample = merge_tencent_features(sample, tencent_risk, 'tc_risk')

## 加载尚为逾期

In [17]:
def merge_shangwei_features(df, shangwei_features, feature_index):
    shangwei_features.columns = pd.MultiIndex.from_product([[feature_index], shangwei_features.columns])
    df = df.merge(shangwei_features, right_on=[(feature_index, 'mobile'), (feature_index, 'back_date')], 
                  left_on=[('sample', 'customer_phone'), ('sample', 'apply_date_key')], how='left').drop_duplicates()
    df = df.drop(columns=[(feature_index, c) for c in ['seq_id', 'mobile', 'idcard', 'name', 'encrypt_type', 'back_date', 'ext_info']], axis=1)
    return df

In [18]:
shangwei_overdue = pd.read_csv("/sda/huweipeng/project/长安银行/data/third_party_data/尚为数据/白鹭报告-逾期风险报告v2+0407swrj.txt", sep="\t", parse_dates=['back_date'])
sample = merge_shangwei_features(sample, shangwei_overdue, 'sw_overdue')
# shangwei_overdue = shangwei_overdue.query('y == "y"').dropna(subset=['overdue_risk_bank_loan_serious_cnt_60d']).drop(columns=['overdue_risk_time_lately_720d', 'y'])
# # sample = sample.merge(shangwei_overdue, right_on=['mobile', 'back_date'], left_on=['customer_phone', 'apply_date_key']).drop_duplicates()
# sample = sample.drop(columns=['seq_id', 'mobile', 'idcard', 'name', 'encrypt_type', 'back_date', 'ext_info'])

## 加载尚为信用评估

In [19]:
shangwei_credit = pd.read_csv("/sda/huweipeng/project/长安银行/data/third_party_data/尚为数据/尚为立信模型-综合信用评估1+0407swrj.txt", sep="\t", parse_dates=['back_date'])
sample = merge_shangwei_features(sample, shangwei_credit, 'sw_credit_eval')
# shangwei_credit = shangwei_credit.query('y == "y"').drop(columns=['y'])[['mobile', 'back_date', 'omriskscoregeneral']]
# sample = sample.merge(shangwei_credit, right_on=['mobile', 'back_date'], left_on=['customer_phone', 'apply_date_key'], how='left')
# sample = sample.drop(columns=['mobile', 'back_date']).drop_duplicates()
# peek(shangwei_credit, 2)

In [20]:
shangwei_credit.columns

MultiIndex([('sw_credit_eval',             'seq_id'),
            ('sw_credit_eval',             'mobile'),
            ('sw_credit_eval',             'idcard'),
            ('sw_credit_eval',               'name'),
            ('sw_credit_eval',       'encrypt_type'),
            ('sw_credit_eval',          'back_date'),
            ('sw_credit_eval',           'ext_info'),
            ('sw_credit_eval', 'omriskscoregeneral'),
            ('sw_credit_eval',                  'y')],
           )

## 加载尚为还款能力

In [21]:
shangwei_repay = pd.read_csv("/sda/huweipeng/project/长安银行/data/third_party_data/尚为数据/翠鸟画像-还款能力v3+0407swrj.txt", sep="\t", parse_dates=['back_date'])
sample = merge_shangwei_features(sample, shangwei_repay, 'sw_repayability')

## 加载尚为收入指数

In [22]:
shangwei_income = pd.read_csv("/sda/huweipeng/project/长安银行/data/third_party_data/尚为数据/翠鸟画像-收入指数v5+0407swrj.txt", sep="\t", parse_dates=['back_date'])
sample = merge_shangwei_features(sample, shangwei_income, 'sw_income')

## 加载尚为履约能力

In [23]:
shangwei_contract = pd.read_csv("/sda/huweipeng/project/长安银行/data/third_party_data/尚为数据/海豚模型-履约能力评估v3+0407swrj.txt", sep="\t", parse_dates=['back_date'])
sample = merge_shangwei_features(sample, shangwei_contract, 'sw_contract')

In [24]:
peek(sample, 2)

(10047, 1989)
                             sample                                  \
                     customer_phone apply_date_key order_status_key   
0  cc92befb0e0b9f7ef4736f0fabc86821     2023-02-22                3   
1  057331edd64b9002ddfc084e2d09b441     2023-03-25                2   

          br_duotou                                            \
  flag_applyloanstr als_d7_id_pdl_allnum als_d7_id_pdl_orgnum   
0                 1                  NaN                  NaN   
1                 1                  NaN                  NaN   

                                                                    \
  als_d7_id_caon_allnum als_d7_id_caon_orgnum als_d7_id_rel_allnum   
0                   NaN                   NaN                  NaN   
1                   NaN                   NaN                  NaN   

                                                                      \
  als_d7_id_rel_orgnum als_d7_id_caoff_allnum als_d7_id_caoff_orgnum   
0              

## 加载人口统计学信息

In [25]:
demographic = pd.read_csv("/sda/huweipeng/project/长安银行/data/样本人口学信息.csv")
demographic.columns = pd.MultiIndex.from_product([['demographic'], demographic.columns])
sample = sample.merge(demographic, right_on=[('demographic', 'lower_phone_md5')], left_on=[('sample', 'customer_phone')], how='left')
sample = sample.drop(columns=[('demographic', 'lower_phone_md5')])

## 加载问卷数据

In [26]:
quest = pd.read_csv("/sda/huweipeng/project/长安银行/data/调查问卷数据.csv")
quest = quest.pivot(index='lower_phone_md5', columns='question_subject', values='answer_content').reset_index()
quest.columns = ['customer_phone', 'degree', 'card_num']
quest.columns = pd.MultiIndex.from_product([['quest'], quest.columns])
sample = sample.merge(quest, left_on=[('sample', 'customer_phone')], right_on=[('quest', 'customer_phone')], how='left')
sample = sample.drop(columns=[('quest', 'customer_phone')])

## 输出

In [27]:
sample.to_csv(config.DATA, index=False)